In [2]:
%pip install qwikidata

Note: you may need to restart the kernel to use updated packages.


In [4]:
from qwikidata.sparql import return_sparql_query_results

Hans  Zimmer  (Q76364)es  un  famoso  compositor  de  muchas  bandas  sonoras  de películas. Vamos a comenzar por averiguar su fecha y lugar de nacimiento

In [5]:


sparql_query = """
SELECT ?birth_date ?place_birth ?place_birthLabel
WHERE
{
  wd:Q76364 wdt:P569 ?birth_date. #fecha nacimiento Hans Zimmer
  wd:Q76364 wdt:P19 ?place_birth. #lugar de nacimiento
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
res = return_sparql_query_results(sparql_query)
res

{'head': {'vars': ['birth_date', 'place_birth', 'place_birthLabel']},
 'results': {'bindings': [{'birth_date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
     'type': 'literal',
     'value': '1957-09-12T00:00:00Z'},
    'place_birth': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q1794'},
    'place_birthLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Frankfurt am Main'}}]}}

<html><head><meta charset="utf-8"></head><body><table><thead><tr><th>birth_date</th><th>place_birthLabel</th></tr></thead><tbody><tr><td>1957-09-12T00:00:00Z</td><td>Fráncfort del Meno</td></tr></tbody></table></body></html>

A continuación,vamos  a  averiguar  los  instrumentos  que  toca.  Queremos  obtener  los resultados ordenados alfabéticamente por el nombre del instrumento. 

In [6]:
sparql_query = """
SELECT ?instrumentLabel 
WHERE
{
  wd:Q76364 wdt:P1303 ?instrument
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?instrumentLabel)

"""
res = return_sparql_query_results(sparql_query)
res

{'head': {'vars': ['instrumentLabel']},
 'results': {'bindings': [{'instrumentLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'banjo'}},
   {'instrumentLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'guitar'}},
   {'instrumentLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'keyboard instrument'}},
   {'instrumentLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'piano'}},
   {'instrumentLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'synthesizer'}}]}}

<html><head><meta charset="utf-8"></head><body><table><thead><tr><th>instrumentLabel</th></tr></thead><tbody><tr><td>banjo</td></tr><tr><td>guitarra</td></tr><tr><td>piano</td></tr><tr><td>sintetizador</td></tr><tr><td>teclado</td></tr></tbody></table></body></html>

¿Cuales de los instrumentos que toca son instrumentos de cuerda? Ten en cuenta que los  instrumentos  pueden  ser  indirectamente  de  cuerda  a  través  de  varias  relaciones subclase. 

In [8]:
sparql_query = """
SELECT ?instrumentLabel 
WHERE
{
  wd:Q76364 wdt:P1303 ?instrument.
  ?instrument wdt:P279* wd:Q1798603.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?instrumentLabel)

"""
res = return_sparql_query_results(sparql_query)
res

{'head': {'vars': ['instrumentLabel']},
 'results': {'bindings': [{'instrumentLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'banjo'}},
   {'instrumentLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'guitar'}},
   {'instrumentLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'piano'}}]}}

<html><head><meta charset="utf-8"></head><body><table><thead><tr><th>instrumentLabel</th></tr></thead><tbody><tr><td>banjo</td></tr><tr><td>guitarra</td></tr><tr><td>piano</td></tr></tbody></table></body></html>

Zimmer   ha   recibido   muchas nominaciones   y   premios   a   lo   largo   de   su   carrera. Queremos obtener una lista de nominaciones y para cada una de ellas el trabajo por el cual fue nominado. Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarásentender los prefijos que usa Wikidata. 